<a href="https://colab.research.google.com/github/cadmiiross/EjeRestaurantes/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 1.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8e33587b981b4585dd4679a08ca393426e1fb97359f2555863b894651b408cc2
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Leer el csv
Crear un objeto de tipo DataFrameReader usando la propiedad spark.read
Crear un objeto de lectura especificando los headers.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

prices = spark.read.csv("prices.csv")
prices.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|      _c0|        _c1|         _c2|    _c3|  _c4|     _c5|     _c6|       _c7|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|03/02/2019|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|28/01/2019|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|   30.00|06/02/2019|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|15/02/2019|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|07/02/2019|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|01/02/2019|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|01/02/2019|
+---------+-----------+------------+-------+-----+--------+--------+----------+



In [3]:
prices = spark.read.options(header="true").csv("prices.csv")
prices.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|03/02/2019|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|28/01/2019|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|   30.00|06/02/2019|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|15/02/2019|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|07/02/2019|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|01/02/2019|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|01/02/2019|
+---------+-----------+------------+-------+-----+--------+--------+----------+



Mostrar los tipos de datos que tiene el dataset

In [4]:
from pprint import pprint
pprint(prices.dtypes)

[('store', 'string'),
 ('countrycode', 'string'),
 ('brand', 'string'),
 ('model', 'string'),
 ('price', 'string'),
 ('currency', 'string'),
 ('quantity', 'string'),
 ('date', 'string')]


Definir un esquema de tipos de datos adecuado

In [5]:
from pyspark.sql.types import *
schema = StructType([StructField("store", StringType(), nullable=False), 
                     StructField("countrycode", StringType(), nullable=False), 
                     StructField("brand", StringType(), nullable=False),
                     StructField("model", StringType(), nullable=False),
                     StructField("price", FloatType(), nullable=False),
                     StructField("currency", StringType(), nullable=True),
                     StructField("quantity", IntegerType(), nullable=True),
                     StructField("date", DateType(), nullable=False),])


prices = spark.read.schema(schema).options(header="true").csv("prices.csv")
pprint(prices.dtypes)
prices.show()

[('store', 'string'),
 ('countrycode', 'string'),
 ('brand', 'string'),
 ('model', 'string'),
 ('price', 'float'),
 ('currency', 'string'),
 ('quantity', 'int'),
 ('date', 'date')]
+---------+-----------+------------+-------+-----+--------+--------+----+
|    store|countrycode|       brand|  model|price|currency|quantity|date|
+---------+-----------+------------+-------+-----+--------+--------+----+
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|null|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|null|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|    null|null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|null|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|null|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|null|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|null|
+---------+-----------+------------+-------+-----+--------+--------+----+



Eliminar las filas erroneas

In [6]:
pricesRowsRemoved = spark.read.schema(schema).options(header="true", mode="DROPMALFORMED").csv("prices.csv")
pricesRowsRemoved.show()

+-----+-----------+-----+-----+-----+--------+--------+----+
|store|countrycode|brand|model|price|currency|quantity|date|
+-----+-----------+-----+-----+-----+--------+--------+----+
+-----+-----------+-----+-----+-----+--------+--------+----+



In [7]:
pricesRowsRemoved = spark.read.schema(schema).options(header="true", mode="DROPMALFORMED", dateformat = "dd/MM/yyyy").csv("prices.csv")
pprint(pricesRowsRemoved.dtypes)
pricesRowsRemoved.show()

[('store', 'string'),
 ('countrycode', 'string'),
 ('brand', 'string'),
 ('model', 'string'),
 ('price', 'float'),
 ('currency', 'string'),
 ('quantity', 'int'),
 ('date', 'date')]
+--------+-----------+------------+-------+-----+--------+--------+----------+
|   store|countrycode|       brand|  model|price|currency|quantity|      date|
+--------+-----------+------------+-------+-----+--------+--------+----------+
|    Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|2019-02-03|
|Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|2019-02-15|
|   Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|      DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|2019-02-01|
|      DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|2019-02-01|
+--------+-----------+------------+-------+-----+--------+--------+----------+



Rellenar valores vacios

In [8]:
prices3 = spark.read.options(header="true").schema(schema).csv("prices3.csv") #dateFormat="dd/MM/yyyy"
prices3.show()
prices3.fillna(40, subset=["quantity"]).show()
#varios datos vacios---> prices3.fillna({"quantity":40, "date":01/03/1990}).show()

+---------+-----------+------------+-------+-----+--------+--------+----+
|    store|countrycode|       brand|  model|price|currency|quantity|date|
+---------+-----------+------------+-------+-----+--------+--------+----+
|     Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|    null|null|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|null|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|      30|null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|    null|null|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|null|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|    null|null|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|    null|null|
+---------+-----------+------------+-------+-----+--------+--------+----+

+---------+-----------+------------+-------+-----+--------+--------+----+
|    store|countrycode|       brand|  model|price|currency|quantity|date|
+---------+-----------+------------+-

Utilizar condiciones para afectar valores especificos

In [9]:
prices4 = spark.read.schema(schema).options(header="true", dateFormat="dd/MM/yyyy").csv("prices4.csv") 
prices4.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months| 6.87|     EUR|      40|2025-02-03|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|      30|      null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|2019-02-15|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|2025-02-01|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|2019-02-01|
+---------+-----------+------------+-------+-----+--------+--------+----------+



In [10]:
from pyspark.sql.functions import col, when
from datetime import date
one_year_from_now = date.today().replace(year=date.today().year + 1) #fecha actual con un año posterior
new_frame = prices4.withColumn("date", when(col("date")> one_year_from_now, None).otherwise(col("date")))
new_frame.show()

+---------+-----------+------------+-------+-----+--------+--------+----------+
|    store|countrycode|       brand|  model|price|currency|quantity|      date|
+---------+-----------+------------+-------+-----+--------+--------+----------+
|     Aldi|         BE|Diapers-R-Us|6months| 6.87|     EUR|      40|      null|
| Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|2019-01-28|
|Carrefour|         FR|     Nappy-k|2months|  5.7|     EUR|      30|      null|
| Kruidvat|         NL|     Nappy-k|2months|  5.6|     EUR|      40|2019-02-15|
|    Tesco|        IRL|     Pampers|3months|  6.3|     EUR|      35|2019-02-07|
|       DM|         DE|     Huggies|newborn|  6.8|     EUR|      40|      null|
|       DM|         AT|     Huggies|newborn|  7.2|     EUR|      40|2019-02-01|
+---------+-----------+------------+-------+-----+--------+--------+----------+



filtrar y ordenar filas

In [13]:
from pyspark.sql.functions import col
pricesBE = prices.filter(col("countrycode")=="BE").orderBy(col("date"))
pricesBE.show()

+--------+-----------+------------+-------+-----+--------+--------+----+
|   store|countrycode|       brand|  model|price|currency|quantity|date|
+--------+-----------+------------+-------+-----+--------+--------+----+
|    Aldi|         BE|Diapers-R-Us|6months|  6.8|     EUR|      40|null|
|Kruidvat|         BE|     Nappy-k|2months|  4.8|     EUR|      30|null|
+--------+-----------+------------+-------+-----+--------+--------+----+



Seleccionar algunas columnas y renombrarlas

In [14]:
prices.select(col("store"), col("brand").alias("brandname")).show()

+---------+------------+
|    store|   brandname|
+---------+------------+
|     Aldi|Diapers-R-Us|
| Kruidvat|     Nappy-k|
|Carrefour|     Nappy-k|
| Kruidvat|     Nappy-k|
|    Tesco|     Pampers|
|       DM|     Huggies|
|       DM|     Huggies|
+---------+------------+



Eliminar filas repetitivas

In [16]:
prices.select(col("store"), col("brand").alias("brandname")).distinct().show()

+---------+------------+
|    store|   brandname|
+---------+------------+
|       DM|     Huggies|
| Kruidvat|     Nappy-k|
|Carrefour|     Nappy-k|
|     Aldi|Diapers-R-Us|
|    Tesco|     Pampers|
+---------+------------+



Agrupar

In [17]:
(prices.groupBy(col("brand")).mean("price")).show()

+------------+------------------+
|       brand|        avg(price)|
+------------+------------------+
|Diapers-R-Us| 6.800000190734863|
|     Pampers| 6.300000190734863|
|     Huggies|               7.0|
|     Nappy-k|5.3666666348775225|
+------------+------------------+



Agregacion

In [18]:
from pyspark.sql.functions import avg, count,stddev_samp, max as sfmax
(prices.groupBy(col("brand")).agg(avg("price").alias("avg_price"),count("brand").alias("items"))).show()

+------------+------------------+-----+
|       brand|         avg_price|items|
+------------+------------------+-----+
|Diapers-R-Us| 6.800000190734863|    1|
|     Pampers| 6.300000190734863|    1|
|     Huggies|               7.0|    2|
|     Nappy-k|5.3666666348775225|    3|
+------------+------------------+-----+



In [19]:
ratings = spark.read.options(header="true").csv("ratings.csv")
ratings.show()

+------------+-------+---------------+-------+
|       brand|  model|absorption_rate|comfort|
+------------+-------+---------------+-------+
|Diapers-R-Us|6months|              2|      3|
|     Nappy-k|2months|              3|      4|
|     Pampers|3months|              4|      4|
|     Huggies|newborn|              3|      5|
+------------+-------+---------------+-------+



In [20]:
ratings_with_prices = ratings.join(prices, ["brand", "model"])
ratings_with_prices.show()

+------------+-------+---------------+-------+---------+-----------+-----+--------+--------+----+
|       brand|  model|absorption_rate|comfort|    store|countrycode|price|currency|quantity|date|
+------------+-------+---------------+-------+---------+-----------+-----+--------+--------+----+
|Diapers-R-Us|6months|              2|      3|     Aldi|         BE|  6.8|     EUR|      40|null|
|     Nappy-k|2months|              3|      4| Kruidvat|         BE|  4.8|     EUR|      30|null|
|     Nappy-k|2months|              3|      4|Carrefour|         FR|  5.7|     EUR|    null|null|
|     Nappy-k|2months|              3|      4| Kruidvat|         NL|  5.6|     EUR|      40|null|
|     Pampers|3months|              4|      4|    Tesco|        IRL|  6.3|     EUR|      35|null|
|     Huggies|newborn|              3|      5|       DM|         DE|  6.8|     EUR|      40|null|
|     Huggies|newborn|              3|      5|       DM|         AT|  7.2|     EUR|      40|null|
+------------+------